**Harvard University**<br/>
**Fall 2024**<br/>
**Instructors**: Pavlos Protopapas, Natesh Pillai, and Chris Gumb

In [1]:
# RUN THIS CELL
import itertools
import math
import time

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_validate, cross_val_score, cross_val_predict

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegressionCV

from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

# pandas tricks for better display
pd.options.display.max_columns = 50  
pd.options.display.max_rows = 500     
pd.options.display.max_colwidth = 100
pd.options.display.precision = 3

<a id="contents"></a>

## Notebook contents

- [**Overview and data description**](#intro)



In [2]:
# Pulling data from csv
obesity = pd.read_csv('data/Obesity Prevalence.csv')
mortality = pd.read_csv('data/Mortality-Database-Cardiovascular-Diseases-Oct-29-2024.csv')

obesity.drop(['Language', 'DateModified', 'FactComments', 'FactValueTranslationID', 'Dim3 type', 'Dim3', 'Dim3ValueCode', 'DataSourceDimValueCode', 'DataSource','FactValueNumericPrefix', 'FactValueUoM', 'FactValueNumericLowPrefix', 'FactValueNumericHighPrefix'],axis=1)

print(obesity.dtypes)
print(mortality.dtypes)

IndicatorCode                  object
Indicator                      object
ValueType                      object
ParentLocationCode             object
ParentLocation                 object
Location type                  object
SpatialDimValueCode            object
Location                       object
Period type                    object
Period                          int64
IsLatestYear                     bool
Dim1 type                      object
Dim1                           object
Dim1ValueCode                  object
Dim2 type                      object
Dim2                           object
Dim2ValueCode                  object
Dim3 type                     float64
Dim3                          float64
Dim3ValueCode                 float64
DataSourceDimValueCode        float64
DataSource                    float64
FactValueNumericPrefix        float64
FactValueNumeric              float64
FactValueUoM                  float64
FactValueNumericLowPrefix     float64
FactValueNum

<a id="intro"></a>

## Overview and data description

[Return to contents](#contents)

### Data

#### Obesity Prevalence Data

- **IndicatorCode**: a unique identifier for the overall data 
- **Indicator**: Description of overall data
- **ValueType**: data type
- **ParentLocationCode**: Region code (i.e. AFR)
- **ParentLocation**: Region name (i.e. Africa)
- **Location type**: Location type (i.e. country, territory)
- **SpatialDimValueCode**: Location code (i.e. ETH)
- **Location**: Location name (i.e. Ethiopia)
- **Period type**: Time period type (i.e. Year)
- **Period**: Specific time period
- **IsLatestYear**: Boolean for latest data
- **Dim1 type**: Sex
- **Dim1**: Gender
- **Dim1ValueCode**: Gender code
- **Dim2 type**: Age group
- **Dim2**: Specific age group
- **Dim2ValueCode**: Age code
- **FactValueNumeric**: Estimated obesity percentage
- **FactValueNumericLow**: Lower bound on obesity percentage
- **FactValueNumericHigh**: Upper bound on obesity percentage
- **Value**: Estimated obesity percentage [Lower bound - Upper bound] rounded to the nearest tenth

#### Mortality Data
- **Region Code**: Region ID
- **Region Name**: Region name
- **Country Code**: Country ID
- **Country Name**: Country Name
- **Year**: Year data was collected
- **Sex**: Sex of partipants
- **Age group code**: Age ID
- **Age Group**: Age in [lower-upper] format
- **Number**: Absolute number of deaths
- **Percentage of cause-specific deaths out of total deaths**: 
- **"Age-standardized death rate per 100 000 standard population"**:
- **"Death rate per 100 000 population"**:

#### Cardiovascular Mortality
- **Region Code**: Region ID
- **Region Name**: Region name
- **Country Code**: Country ID
- **Country Name**: Country Name
- **Year**: Year data was collected
- **Sex**: Sex of partipants
- **Age group code**: Age ID
- **Age Group**: Age in [lower-upper] format
- **Number**: Absolute number of deaths
- **Percentage of cause-specific deaths out of total deaths**: 
- **"Age-standardized death rate per 100 000 standard population"**:
- **"Death rate per 100 000 population"**:

## Finalized Research Question

### How does the prevalence of obesity correlate with overall mortality rates across different regions, sexes, and age groups over time?

### Data Cleaning

In [5]:
df = pd.read_csv(
    "data/cleaned_mortality_data.csv",
    index_col=False,
    na_values=["", " "],  # Treat empty values as NaN
    skipinitialspace=True,  # Ignore extra spaces after commas
    on_bad_lines="warn"  # Skip problematic rows with warnings
)

In [6]:
df.head()

,Region Code,Region Name,Country Code,Country Name,Year,Sex,Age group code,Age Group,Number,Percentage of cause-specific deaths out of total deaths,Age-standardized death rate per 100 000 standard population,Death rate per 100 000 population
0,EU,Europe,ALB,Albania,1987,All,Age_unknown,[Unknown],0.0,NaN,NaN,NaN
1,EU,Europe,ALB,Albania,1987,All,Age85_over,[85+],1211.0,54.403,NaN,12110.000
2,EU,Europe,ALB,Albania,1987,All,Age80_84,[80-84],955.0,58.267,NaN,5753.012
3,EU,Europe,ALB,Albania,1987,All,Age75_79,[75-79],1178.0,55.697,NaN,3824.675
4,EU,Europe,ALB,Albania,1987,All,Age70_74,[70-74],969.0,53.654,NaN,2375.000


In [15]:
# Drop columns with more than 50% NaN values
threshold = 0.5  # Keep columns with at least 50% non-NaN values
df_cleaned = df.dropna(thresh=int(threshold * len(df)), axis=1)

# Remove rows where 'Age group code' is 'Age_unknown'
if 'Age group code' in df_cleaned.columns:
    df_cleaned = df_cleaned[df_cleaned['Age group code'] != 'Age_unknown']

# Limit the dataset to specific countries for reliable data
country_list = [
    'United States', 'Canada', 'Germany', 'France', 'United Kingdom',
    'Netherlands', 'Norway', 'Japan', 'South Korea',
    'Australia', 'New Zealand'
]
if 'Country Name' in df_cleaned.columns:
    df_filtered = df_cleaned[df_cleaned['Country Name'].isin(country_list)]
else:
    df_filtered = df_cleaned  # If 'Country Name' column doesn't exist, skip this step

# Display the cleaned and filtered dataframe
df_filtered


,Region Code,Region Name,Country Code,Country Name,Year,Sex,Age group code,Age Group,Number,Percentage of cause-specific deaths out of total deaths,Death rate per 100 000 population
9009,OA,Oceania,AUS,Australia,1950,All,Age_all,[All],37867.0,48.431,462.995
9011,OA,Oceania,AUS,Australia,1950,All,Age00,[0],12.0,0.257,6.490
9012,OA,Oceania,AUS,Australia,1950,All,Age01_04,[1-4],15.0,1.335,2.130
9013,OA,Oceania,AUS,Australia,1950,All,Age05_09,[5-9],28.0,5.333,3.979
9014,OA,Oceania,AUS,Australia,1950,All,Age10_14,[10-14],32.0,8.399,5.488
...,...,...,...,...,...,...,...,...,...,...,...
310228,OA,Oceania,AUS,Australia,2018,Female,Age20_24,[20-24],10.0,5.917,1.183
310229,OA,Oceania,AUS,Australia,2018,Female,Age25_29,[25-29],11.0,4.977,1.180
310230,OA,Oceania,AUS,Australia,2018,Female,Age30_34,[30-34],33.0,10.217,3.519
310231,OA,Oceania,AUS,Australia,2018,Female,Age35_39,[35-39],49.0,9.839,5.649


In [17]:
# Save the DataFrame to a CSV file
output_file_path = "data/cleaned_mortality_data.csv"  # Specify your desired file name and path
df_filtered.to_csv(output_file_path, index=False)

print(f"DataFrame saved to {output_file_path}")


DataFrame saved to data/cleaned_mortality_data.csv
